# Anotación de entidades mencionadas: Trafilatura, spaCy e LabelStudio

## Trafilatura

Ferramenta deseñada para obter datos textuais a partir da web, e que se pode usar como programa desde a liña de comandos ou como módulo de Python.

https://trafilatura.readthedocs.io/en/latest/

In [ ]:
!pip install trafilatura

In [ ]:
!trafilatura -h

Lista de URLs para descarregar:

In [ ]:
!cat list.txt

In [ ]:
!trafilatura -i list.txt -o texts